In [ ]:
from ultralytics import YOLO

# <div dir="rtl">עבודה עם YOLO כולל דוגמאות</div>

#### <div dir="rtl">טעינת המודל = https://docs.ultralytics.com/models/yolo11/</div>
<div dir="rtl">שימו לב נעשה כאן שימוש במודל YOLO11n לדיוק גבוה יותר עברו למודלים חזקים יותר כגון מודל ה small medium וכדו</div>

In [ ]:
model = YOLO('yolo11n.pt')

#### משימה ראשונה בדיקה עם יש אדם בתמונה או לא


In [ ]:
results = model.predict(r"<path>")
for r in results:
    labels = [model.names[int(c)] for c in r.boxes.cls]
    if "person" in labels:
        print("יש אדם בתמונה")
    else:
        print("אין אדם בתמונה")

In [ ]:
def has_preson(images):

    for img in images:

        results = model.predict(img)
        for r in results:
            labels = [model.names[int(c)] for c in r.boxes.cls]
            print(img)
            if "person" in labels:
                print("יש אדם בתמונה")
            else:
                print("אין אדם בתמונה")

### בדיקת דיוק נתוני מודל

In [ ]:
import os
from ultralytics import YOLO



# הגדר את נתיב התיקיה
folder_path = r"<>"

# שמות המחלקות שאנחנו רוצים לבדוק (לדוגמה: 'person')
target_classes = ['person']

# ספירה
total_images = 0
detected_images = 0

# עבור כל קובץ בתיקייה
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        total_images += 1
        image_path = os.path.join(folder_path, filename)
        results = model(image_path)

        found = False
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for cls_tensor in boxes.cls:
                    class_id = int(cls_tensor.item())
                    class_name = model.names[class_id]
                    if class_name in target_classes:
                        found = True
                        break

        if found:
            detected_images += 1

# הצג אחוז דיוק בסיסי
accuracy = (detected_images / total_images) * 100
print(f"Detected 'person' in {detected_images} out of {total_images} images.")
print(f"Accuracy: {accuracy:.2f}%")


### ספירת כמות אנשים בתמונה


In [ ]:
# count person

persons = model.predict(r"<>")

person_count = 0


for result in persons:
    boxes = result.boxes
    if boxes is not None:
        for cls_tensor in boxes.cls:
            class_id = int(cls_tensor.item())
            class_name = model.names[class_id]
            if class_name == 'person':
                person_count += 1

print(f"Number of people in the image: {person_count}")

persons[0].show()  # Show the first result with detected persons


In [ ]:
for r in results:
    labels = [model.names[int(c)]for c in r.boxes.cls]
labels

In [ ]:
results[0].show()

### סינון פריטים מסויימים

In [ ]:
allowed_classes = ['car']
for result in results:
    boxes = result.boxes
    if boxes is not None:
        keep = []
        for i, cls_tensor in enumerate(boxes.cls):
            class_id = int(cls_tensor.item())
            class_name = model.names[class_id]
            if class_name in allowed_classes:
                keep.append(i)

        # שמור רק את התיבות שאנחנו רוצים
        if keep:
            result.boxes = result.boxes[keep]
        else:
            result.boxes = None  # אין מה להציג

# הצג רק את התיבות המסוננות
results[0].show()

## Person Detect In Video

In [ ]:
import cv2
from ultralytics import YOLO

# טען את המודל

# טען את הווידאו
video_path = r'<path>'
cap = cv2.VideoCapture(video_path)

# קבל את קצב הפריימים לשנייה (FPS)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_index = 0
seconds_with_person = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # שמור את שניית הווידאו הנוכחית
    current_second = int(frame_index / fps)

    # שמור את התמונה לקובץ זמני אם אתה עובד עם מודל YOLO שמקבל path
    results = model(frame)

    # בדוק אם יש אדם בפריים
    person_found = False
    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for cls_tensor in boxes.cls:
                class_id = int(cls_tensor.item())
                class_name = model.names[class_id]
                if class_name == 'person':
                    person_found = True
                    break

    if person_found:
        seconds_with_person.add(current_second)

    frame_index += 1

cap.release()

# הדפסת השניות שבהן הופיע אדם
print("Seconds in which a person was detected:")
for sec in sorted(seconds_with_person):
    print(f"Second: {sec}")


In [ ]:
import cv2
from ultralytics import YOLO
from datetime import datetime
import os

# טען את מודל YOLO


# צור תיקיה לשמירת התמונות אם לא קיימת
output_dir = 'captured_frames'
os.makedirs(output_dir, exist_ok=True)

# התחבר למצלמת המחשב (0 זה המצלמה הראשית)
cap = cv2.VideoCapture(0)

print("⏳ מאזין למצלמה... ילכוד פריימים כשאדם מופיע")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # שלח את הפריים לזיהוי
    results = model(frame)

    person_detected = False

    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for cls_tensor in boxes.cls:
                class_id = int(cls_tensor.item())
                class_name = model.names[class_id]
                if class_name == 'person':
                    person_detected = True
                    break

    if person_detected:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = os.path.join(output_dir, f'person_{timestamp}.jpg')
        cv2.imwrite(filename, frame)
        print(f"📸 אדם זוהה! תמונה נשמרה: {filename}")

    # יציאה על ידי לחיצה על מקש q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### לכידת אדם בודאו

In [ ]:
import cv2
from ultralytics import YOLO
import os
from datetime import datetime

# טען את המודל


# פתח את המצלמה
cap = cv2.VideoCapture(0)

# פרטים טכניים של הווידאו
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS) or 30)

# צור תיקיה
output_dir = 'final_video_output'
os.makedirs(output_dir, exist_ok=True)

# שם הקובץ הסופי
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = os.path.join(output_dir, f'people_detected_{timestamp}.mp4')

# הגדרת ה־VideoWriter (הקלטה לקובץ)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print("🎥 התחברות למצלמה... הקלטת פריימים עם אדם בלבד.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # בצע תחזית עם YOLO
    results = model(frame)
    person_detected = False

    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for cls_tensor in boxes.cls:
                cls_id = int(cls_tensor.item())
                class_name = model.names[cls_id]
                if class_name == 'person':
                    person_detected = True
                    break

    # כתוב פריים לקובץ רק אם זוהה אדם
    if person_detected:
        video_writer.write(frame)
        print("🧍 אדם זוהה — פריים נשמר.")

    # יציאה בלחיצה על 'q'
    cv2.imshow("Live Feed", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# סיום
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"✅ וידאו סופי נשמר: {output_path}")


✅ תרגיל 1: חיתוך ושמירת האובייקטים כקבצים נפרדים

In [ ]:
import cv2
from ultralytics import YOLO
import os

# טען את המודל


# טען תמונה
image_path = r"<>"
image = cv2.imread(image_path)

# הרץ זיהוי
results = model(image)[0]

# תיקייה לשמירת אובייקטים
output_folder = "cropped_objects"
os.makedirs(output_folder, exist_ok=True)

# חיתוך ושמירה
for i, box in enumerate(results.boxes):
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    class_id = int(box.cls[0])
    class_name = model.names[class_id]

    cropped = image[y1:y2, x1:x2]
    filename = f"{output_folder}/{class_name}_{i}.jpg"
    cv2.imwrite(filename, cropped)
    print(f"✔️ נשמר: {filename}")


✅ תרגיל 2: מערכת התראה בזמן אמת לזיהוי אובייקט מסוים (למשל אדם)

In [ ]:
import cv2
from ultralytics import YOLO
# from playsound import playsound  # pip install playsound
import threading

# טען מודל
model = YOLO("yolov8n.pt")
target_class_name = "person"  # עצם שצריך להתריע עליו

# דגל התראה
alert_triggered = False

# פונקציית השמעת צליל פעם אחת בכל זיהוי
# def play_alert():
#     global alert_triggered
#     if not alert_triggered:
#         alert_triggered = True
#         playsound("alert.mp3")  # שים קובץ צליל בשם זה בתיקייה
#         alert_triggered = False



def play_alert():
    global alert_triggered
    if not alert_triggered:
        alert_triggered = True
        # playsound("alert.mp3")  # שים קובץ צליל בשם זה בתיקייה
        print("🔔 התראה: אדם זוהה!")
        alert_triggered = False
# התחברות למצלמה
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    for box in results.boxes:
        class_id = int(box.cls[0])
        class_name = model.names[class_id]

        if class_name == target_class_name:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, class_name, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            threading.Thread(target=play_alert).start()

    cv2.imshow("Detection", frame)
    if cv2.waitKey(1) == 27:  # ESC לסיום
        break

cap.release()
cv2.destroyAllWindows()


### יצירת וידאו רק מפריימים שמופיע בהם אדם

In [ ]:
import cv2
from ultralytics import YOLO
import os
from datetime import datetime

# טען את המודל
model = YOLO('yolo11n.pt')

# פתח את המצלמה
cap = cv2.VideoCapture(0)

# פרטים טכניים של הווידאו
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS) or 30)

# צור תיקיה
output_dir = 'final_video_output'
os.makedirs(output_dir, exist_ok=True)

# שם הקובץ הסופי
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = os.path.join(output_dir, f'people_detected_{timestamp}.mp4')

# הגדרת ה־VideoWriter (הקלטה לקובץ)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print("🎥 התחברות למצלמה... הקלטת פריימים עם אדם בלבד.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # בצע תחזית עם YOLO
    results = model(frame)
    person_detected = False

    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for cls_tensor in boxes.cls:
                cls_id = int(cls_tensor.item())
                class_name = model.names[cls_id]
                if class_name == 'person':
                    person_detected = True
                    break

    # כתוב פריים לקובץ רק אם זוהה אדם
    if person_detected:
        video_writer.write(frame)
        print("🧍 אדם זוהה — פריים נשמר.")

    # יציאה בלחיצה על 'q'
    # cv2.imshow("Live Feed", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

    cv2.waitKey(1)

# סיום
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"✅ וידאו סופי נשמר: {output_path}")


##